# Differentially Located IS's (DLIS)
En este programa analizamos las DLIS encontradas utilizando del [IS-Compare](https://github.com/maurijlozano/ISCompare).
Cepa de Referencia: Tohama///Cepas Query: las 170 cepas del paper 'Genomic Survey of Bordetellla pertussis'.

Este programa se alimenta del archivos 'ISs_Strains_clean.cvs'

En este archivo analizamos las DLIS que estan indicadas como "Verify manually"

---
## Import

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 350)

In [ ]:
df = pd.read_csv('ISs_Strains_clean.csv')

In [ ]:
print(df.shape)
df

---
## Contador de cepas
Las lineas de codigo que estan a continuacion nos permiten contar cuantas cepas estamos analizando

In [ ]:
numero_cepas = df['ID'].nunique()

In [ ]:
print(f' Numero de cepas analizadas: >>>>> {numero_cepas}')

---
## Verify Manualy..... corregir desde aca
En el bloque que sigue a continuacion seleccionamos solo los hits que arrojaron DLIS como resultados.
Notar que hay otros hits que se sugiere revisar en forma manual que NO forman parte de este analisis

In [ ]:
for i in df['Observations'].unique():
    print(f'\n{i}')

In [ ]:
# Nos quedamos solo con las filas que tienen 'Observation == 1.DLIS'
df = df[df['Observations']!='1. DLIS']

In [ ]:
print(df.shape)
df

In [ ]:
# Analizamos que tipo de datos estan aojados en cada columna
df.dtypes 

In [ ]:
# Para poder trabajar con la funcion .groupby() necesitamos que la columna Year sea un object y no int64
# Por lo tanto modificamos el formato de esa columna
df['Year'] = df['Year'].astype(object)


In [ ]:
# Confirmamos la modificacion del tipo de dato
df.dtypes

---
## DLIS por cepa por year

In [ ]:
# La tabla que se muestra a continuacion muestra el numero de DLIS encontradas por el IS-Compare para cada cepa, agrupadas
# segun el anio en que fueron aisladas

df1 = df.groupby(['Year','ID']).count()[['Observations']]

In [ ]:
df1

In [ ]:
# Similar a la anterior, pero ahora tambien agrupamos segun sea que la IS solo esta en Tohama (ref Strain) o solo esta
# en el aislamiento

# IMPORTANTE
# Lo interesante de esta observacion es que pareciera que hay 3 Is's que estaban en Tohama y 
# no estan en los aislamientos clinicos
# utiizamos el SnapGene para confirmar si Tohama tiene una IS481 en esas posiciones
df2 = df.groupby(['Year','ID','Description']).count()[['Observations']]

In [ ]:
df2

---
## IS's only present in Tohama.
En este bloque analizamos aquellas IS's que estan en Tohama pero NO se encuentran en los aislamientos clinicos (post 2000)

In [ ]:
# con este bloque de codigo generamos la df_ref que solo tiene las dlis que estan en Tohama y NO estan en los aislamientos
df_ref = df[df['Description']=='IS only present in ref Strain.']

In [ ]:
print(f'df_ref >>> {df_ref.shape}')
df_ref

In [ ]:
# Analizamos las posiciones en las que estan las ISs que ESTABAN en Tohama y ya NO ESTAN en los aislamientos
df_ref01 = df_ref.groupby(['Insertion_site(REF)']).count()[['Observations']].sort_values('Observations', ascending = False)


In [ ]:
print(f'df_ref >>> {df_ref01.shape}')
df_ref01

# hay un monton de hits que deberiamos verificar!!

---
## IS's only present in  query strains.
En este bloque analizamos aquellas IS's que estan en los aislamientos (post 2000) pero NO se encuentran en Tohama 

In [ ]:
# con este bloque de codigo generamos la df_query que solo tiene las DLIS que estan en los aislamientos 
# y NO estan en Tohama
df_query = df[df['Description']=='IS only present in query Strain.']

In [ ]:
print(f'df_query >>> {df_query.shape}')
df_query

In [ ]:
# Analizamos al lado de que genes estan las ISs que estan los aislamientos y NO estan en Tohama.
df_query01 = df_query.groupby(['Insertion_site(REF)']).count()[['Observations']].sort_values('Observations', ascending = False)


In [ ]:
df_query01.head(50)
# esta tabla tambien habria que tomarse el trabajo de revisar algunas cosas a mano

## Genes INTERRUMPIDOS por IS's
En este bloque buscamos IS's que cayeron en el medio de una secuencia codificante es decir que podrian haber interrumpido 
el gen y evitar su transcripcion. Simil a lo que sucede con algunas cepas prn menos.

nota: este analisis es muy preliminar, hay que pulir esta seleccion

In [ ]:
df_query03 = df_query[df_query['REF_Flank1.locus_tag']==df_query['REF_Flank2.locus_tag']]

In [ ]:
print(df_query03.shape)
df_query03

In [ ]:
df_query04 = df_query03.groupby(['REF_Flank1.locus_tag']).count()[['Observations']].sort_values('Observations', ascending = False)

In [ ]:
df_query04

# recordar que 1054 es la pertactina
# El gen Bp 2611 esta interrumpido en casi todos (143/153!!) los aislamientos clinicos post 2000 (en USA)
#que es la Bp2611? Podria ser un regulador transcripcional!!

# hay 8 cepas en las que BP2611 no esta interrumpida...cuales son? De cuando son?

### Hasta aca llegamos... como seguimos...